### Halı Şube Satış Analiz

In [215]:
import unicodedata
import datetime as dt
import os
import sys
import re
import numpy as np
import pandas as pd
import math
from pathlib import Path
import nltk
from nltk import word_tokenize 
import shutil
from os.path import isfile, join
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import locale
from locale import atof
from geopy.geocoders import Nominatim
from geopy.point import Point
from geopy.exc import GeocoderTimedOut
from functools import reduce

#### Functions

In [216]:
# custom character alphabet for word
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß]+", re.IGNORECASE|re.UNICODE) # German
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓعٮێڔ۶ؚۧۜڤۏإٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ؍ي؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨٹڵڪظٶۭ١ڭەڨحٕ؎ٺڷٰ۪۫ڻڥۛڑڟټآڡغګ؊ّٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفرڇچڝ۴بۈٽڕۡھةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻشڍ؇۵كخ\ا]+", re.IGNORECASE|re.UNICODE) # Arabic

In [217]:
def convert_one_character_letter(text):
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. \n
    convert_one_character_letter(text): text is any string word or sentence.
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    return new_text

In [218]:
def clean_text(text, custom_alp=tr): # for only string in specific language
    '''This function extract custom_alp character from string \n
    clean_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.
    '''
    # text_clean = re.findall(custom_alp, text)
    text_result_list = []
    text_list = re.findall(r'\S+', text)
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_result = " ".join(text_result_list)
    return text_result

In [219]:
def lower_func(text):
    '''This function convert string character to lowercase. \n
    lower_func(text=str): text is any string word or sentence
    '''
    string_lower = str.lower(text)
    return string_lower

In [220]:
def upper_func(text):
    '''This function convert string character to uppercase. \n
    upper_func(text=str): text is any string word or sentence
    '''
    string_upper = str.upper(text)
    return string_upper

In [221]:
def capitalize_func(text):
    '''This function convert string character to capitalize case. \n
    capitalize_func(text=str): text is any string word or sentence
    '''
    string_capitalize = str.capitalize(text)
    return string_capitalize

In [222]:
def title_func(text):
    '''This function convert string character to title as each word start with capital letter. \n
    title_func(text=str): text is any string word or sentence
    '''
    string_title = str.title(text)
    return string_title

In [223]:
def character_clean_lower_text(text, custom_alp=tr): # for only string in specific language
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. And also extract custom_alp character from string and convert to lowercase. \n
    character_clean_lower_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.  
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    text_result_list = []
    text_list = re.findall(r'\S+', new_text)  # like as word tokenize
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_join = " ".join(text_result_list)
    text_result = str.lower(text_join)
    return text_result

In [224]:
def convert_to_float(value):
    '''This function convert string numeric value to float type. Like as 1.200,15
    convert_to_float(value): value is a string numeric value
    '''
    value = value.replace(".","").replace(",",".")
    return float(value)

In [225]:
def whitespace_del(text):
    '''This function provides removing left right white space.\n
    whitespace_del(text): text is a string value as word or sentence.
    '''
    text_var = str(text)
    text_var = text_var.strip()  # rstrip lstrip
    return text_var

#### Read Data

In [226]:
variable_date = "24.03.2023-31.08.2024"
input_path = fr"C:\Users\user\Desktop\Data Analysis\TurkuaHome\Sales_Report\Data\Satış\{variable_date}"
output_path = fr"C:\Users\user\Downloads"

data_file = fr"Novo_Halı_Satış ({variable_date})"

In [227]:
df_sales = pd.read_excel(fr"{input_path}\{data_file}.xlsx")
df_sales

,Cari Ünvan,Kodu,Stok Adı,Şehir,Satış Sorumlusu,Tarih,Miktar m2,Tutar TL
0,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,2.16,347.760
1,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,1.00,178.000
2,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSTANBUL,İSMAİL KURT,2023-04-04,24.72,3979.920
3,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,4.89,860.640
4,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSTANBUL,İSMAİL KURT,2023-04-04,6.00,946.200
...,...,...,...,...,...,...,...,...
39635,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,ANGORA 30 MM SUED TABAN,BOLU,CAN AHMET VURAL,2024-08-30,1.35,784.224
39636,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,BOLU,CAN AHMET VURAL,2024-08-30,3.36,1147.608
39637,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,BOLU,CAN AHMET VURAL,2024-08-30,5.60,1912.680
39638,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,MİRA 1300 GR KAYMAZ TABAN PAMUK POLYESTER ŞÖNİL,BOLU,CAN AHMET VURAL,2024-08-30,3.68,1189.341


In [228]:
df_city_location = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Geo Location\Master\Entegra City Location.xlsx")
df_city_location.drop("id", axis=1, inplace=True)
df_city_location

,city,country,invoice_city,latitude,longitude
0,Adana,Turkey,Adana,37.143800,35.498409
1,Adiyaman,Turkey,Adıyaman,37.763750,38.276348
2,Adiyaman,Turkey,Adiyaman,37.763750,38.276348
3,Afyonkarahisar,Turkey,Afyonkarahisar,38.685273,30.642741
4,Afyonkarahisar,Turkey,Afyon,38.685273,30.642741
...,...,...,...,...,...
262,Yanbu,Saudi Arabia,Yanbu,24.094893,38.070525
263,Az Zulfi,Saudi Arabia,Zulfi,26.311619,44.832951
264,Qairawan,Tunisia,Qairawan,35.673549,10.098636
265,Al Khodh,Oman,Al Khodh,23.565997,58.118169


#### Analysis

In [229]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39640 entries, 0 to 39639
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Cari Ünvan       39640 non-null  object        
 1   Kodu             39625 non-null  object        
 2   Stok Adı         39640 non-null  object        
 3   Şehir            39362 non-null  object        
 4   Satış Sorumlusu  39640 non-null  object        
 5   Tarih            39640 non-null  datetime64[ns]
 6   Miktar m2        39640 non-null  float64       
 7   Tutar TL         39640 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 2.4+ MB


In [230]:
df_sales.columns = [x.lower() for x in df_sales.columns]
df_sales.columns = [x.strip() for x in df_sales.columns]
df_sales.rename(columns={"stok adı":"ürün"}, inplace=True)
df_sales

,cari ünvan,kodu,ürün,şehir,satış sorumlusu,tarih,miktar m2,tutar tl
0,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,2.16,347.760
1,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,1.00,178.000
2,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSTANBUL,İSMAİL KURT,2023-04-04,24.72,3979.920
3,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,4.89,860.640
4,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSTANBUL,İSMAİL KURT,2023-04-04,6.00,946.200
...,...,...,...,...,...,...,...,...
39635,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,ANGORA 30 MM SUED TABAN,BOLU,CAN AHMET VURAL,2024-08-30,1.35,784.224
39636,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,BOLU,CAN AHMET VURAL,2024-08-30,3.36,1147.608
39637,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,BOLU,CAN AHMET VURAL,2024-08-30,5.60,1912.680
39638,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,MİRA 1300 GR KAYMAZ TABAN PAMUK POLYESTER ŞÖNİL,BOLU,CAN AHMET VURAL,2024-08-30,3.68,1189.341


In [231]:
df_sales.select_dtypes(include='object')

,cari ünvan,kodu,ürün,şehir,satış sorumlusu
0,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT
1,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT
2,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSTANBUL,İSMAİL KURT
3,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT
4,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSTANBUL,İSMAİL KURT
...,...,...,...,...,...
39635,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,ANGORA 30 MM SUED TABAN,BOLU,CAN AHMET VURAL
39636,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,BOLU,CAN AHMET VURAL
39637,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,BOLU,CAN AHMET VURAL
39638,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,MİRA 1300 GR KAYMAZ TABAN PAMUK POLYESTER ŞÖNİL,BOLU,CAN AHMET VURAL


In [232]:
object_columns = df_sales.select_dtypes(include='object').columns
object_columns

Index(['cari ünvan', 'kodu', 'ürün', 'şehir', 'satış sorumlusu'], dtype='object')

In [233]:
for column in object_columns:
    df_sales[f"{column}"] = df_sales[f"{column}"].apply(lambda x: convert_one_character_letter(x))
    df_sales[f"{column}"] = df_sales[f"{column}"].apply(lambda x: x.strip())

df_sales

,cari ünvan,kodu,ürün,şehir,satış sorumlusu,tarih,miktar m2,tutar tl
0,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,2.16,347.760
1,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,1.00,178.000
2,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSTANBUL,İSMAİL KURT,2023-04-04,24.72,3979.920
3,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,4.89,860.640
4,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSTANBUL,İSMAİL KURT,2023-04-04,6.00,946.200
...,...,...,...,...,...,...,...,...
39635,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,ANGORA 30 MM SUED TABAN,BOLU,CAN AHMET VURAL,2024-08-30,1.35,784.224
39636,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,BOLU,CAN AHMET VURAL,2024-08-30,3.36,1147.608
39637,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,BOLU,CAN AHMET VURAL,2024-08-30,5.60,1912.680
39638,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,MİRA 1300 GR KAYMAZ TABAN PAMUK POLYESTER ŞÖNİL,BOLU,CAN AHMET VURAL,2024-08-30,3.68,1189.341


In [234]:
test = "ali ata bak"

In [235]:
test.split(' ', 1)[0]

'ali'

In [236]:
df_sales["model"] = df_sales["ürün"].apply(lambda x: x.split(' ', 1)[0])
df_sales["model"] = df_sales["model"].apply(lambda x: x.strip())
df_sales

,cari ünvan,kodu,ürün,şehir,satış sorumlusu,tarih,miktar m2,tutar tl,model
0,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,2.16,347.760,NEPTÜN
1,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,1.00,178.000,NEPTÜN
2,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSTANBUL,İSMAİL KURT,2023-04-04,24.72,3979.920,ROMA
3,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,4.89,860.640,NEPTÜN
4,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSTANBUL,İSMAİL KURT,2023-04-04,6.00,946.200,ROMA
...,...,...,...,...,...,...,...,...,...
39635,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,ANGORA 30 MM SUED TABAN,BOLU,CAN AHMET VURAL,2024-08-30,1.35,784.224,ANGORA
39636,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,BOLU,CAN AHMET VURAL,2024-08-30,3.36,1147.608,NEPTÜN
39637,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,BOLU,CAN AHMET VURAL,2024-08-30,5.60,1912.680,NEPTÜN
39638,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,MİRA 1300 GR KAYMAZ TABAN PAMUK POLYESTER ŞÖNİL,BOLU,CAN AHMET VURAL,2024-08-30,3.68,1189.341,MİRA


In [237]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39640 entries, 0 to 39639
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   cari ünvan       39640 non-null  object        
 1   kodu             39640 non-null  object        
 2   ürün             39640 non-null  object        
 3   şehir            39640 non-null  object        
 4   satış sorumlusu  39640 non-null  object        
 5   tarih            39640 non-null  datetime64[ns]
 6   miktar m2        39640 non-null  float64       
 7   tutar tl         39640 non-null  float64       
 8   model            39640 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 2.7+ MB


In [238]:
df_city_location

,city,country,invoice_city,latitude,longitude
0,Adana,Turkey,Adana,37.143800,35.498409
1,Adiyaman,Turkey,Adıyaman,37.763750,38.276348
2,Adiyaman,Turkey,Adiyaman,37.763750,38.276348
3,Afyonkarahisar,Turkey,Afyonkarahisar,38.685273,30.642741
4,Afyonkarahisar,Turkey,Afyon,38.685273,30.642741
...,...,...,...,...,...
262,Yanbu,Saudi Arabia,Yanbu,24.094893,38.070525
263,Az Zulfi,Saudi Arabia,Zulfi,26.311619,44.832951
264,Qairawan,Tunisia,Qairawan,35.673549,10.098636
265,Al Khodh,Oman,Al Khodh,23.565997,58.118169


In [239]:
df_sales_location = pd.merge(df_sales, df_city_location, how="left", left_on=df_sales["şehir"].str.capitalize(), right_on=df_city_location["invoice_city"].str.capitalize())
df_sales_location

,key_0,cari ünvan,kodu,ürün,şehir,satış sorumlusu,tarih,miktar m2,tutar tl,model,city,country,invoice_city,latitude,longitude
0,İstanbul,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,2.16,347.760,NEPTÜN,Istanbul,Turkey,İstanbul,41.006381,28.975872
1,İstanbul,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,1.00,178.000,NEPTÜN,Istanbul,Turkey,İstanbul,41.006381,28.975872
2,İstanbul,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSTANBUL,İSMAİL KURT,2023-04-04,24.72,3979.920,ROMA,Istanbul,Turkey,İstanbul,41.006381,28.975872
3,İstanbul,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSTANBUL,İSMAİL KURT,2023-04-04,4.89,860.640,NEPTÜN,Istanbul,Turkey,İstanbul,41.006381,28.975872
4,İstanbul,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSTANBUL,İSMAİL KURT,2023-04-04,6.00,946.200,ROMA,Istanbul,Turkey,İstanbul,41.006381,28.975872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39635,Bolu,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,ANGORA 30 MM SUED TABAN,BOLU,CAN AHMET VURAL,2024-08-30,1.35,784.224,ANGORA,Bolu,Turkey,Bolu,40.621210,31.646026
39636,Bolu,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,BOLU,CAN AHMET VURAL,2024-08-30,3.36,1147.608,NEPTÜN,Bolu,Turkey,Bolu,40.621210,31.646026
39637,Bolu,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,BOLU,CAN AHMET VURAL,2024-08-30,5.60,1912.680,NEPTÜN,Bolu,Turkey,Bolu,40.621210,31.646026
39638,Bolu,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,MİRA 1300 GR KAYMAZ TABAN PAMUK POLYESTER ŞÖNİL,BOLU,CAN AHMET VURAL,2024-08-30,3.68,1189.341,MİRA,Bolu,Turkey,Bolu,40.621210,31.646026


In [240]:
df_sales_select = df_sales_location.drop(["key_0","invoice_city","şehir"], axis=1)
df_sales_select

,cari ünvan,kodu,ürün,satış sorumlusu,tarih,miktar m2,tutar tl,model,city,country,latitude,longitude
0,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSMAİL KURT,2023-04-04,2.16,347.760,NEPTÜN,Istanbul,Turkey,41.006381,28.975872
1,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSMAİL KURT,2023-04-04,1.00,178.000,NEPTÜN,Istanbul,Turkey,41.006381,28.975872
2,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSMAİL KURT,2023-04-04,24.72,3979.920,ROMA,Istanbul,Turkey,41.006381,28.975872
3,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,İSMAİL KURT,2023-04-04,4.89,860.640,NEPTÜN,Istanbul,Turkey,41.006381,28.975872
4,MİRZA KİLİM LİMİTED ŞİRKETİ,120.01.076,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",İSMAİL KURT,2023-04-04,6.00,946.200,ROMA,Istanbul,Turkey,41.006381,28.975872
...,...,...,...,...,...,...,...,...,...,...,...,...
39635,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,ANGORA 30 MM SUED TABAN,CAN AHMET VURAL,2024-08-30,1.35,784.224,ANGORA,Bolu,Turkey,40.621210,31.646026
39636,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,CAN AHMET VURAL,2024-08-30,3.36,1147.608,NEPTÜN,Bolu,Turkey,40.621210,31.646026
39637,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,CAN AHMET VURAL,2024-08-30,5.60,1912.680,NEPTÜN,Bolu,Turkey,40.621210,31.646026
39638,HİLMİ YİĞİT-YİĞİT BÜNYAN HALI,120.01.351,MİRA 1300 GR KAYMAZ TABAN PAMUK POLYESTER ŞÖNİL,CAN AHMET VURAL,2024-08-30,3.68,1189.341,MİRA,Bolu,Turkey,40.621210,31.646026


In [241]:
df_sales_select.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39640 entries, 0 to 39639
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   cari ünvan       39640 non-null  object        
 1   kodu             39640 non-null  object        
 2   ürün             39640 non-null  object        
 3   satış sorumlusu  39640 non-null  object        
 4   tarih            39640 non-null  datetime64[ns]
 5   miktar m2        39640 non-null  float64       
 6   tutar tl         39640 non-null  float64       
 7   model            39640 non-null  object        
 8   city             39508 non-null  object        
 9   country          39508 non-null  object        
 10  latitude         39508 non-null  float64       
 11  longitude        39508 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(7)
memory usage: 3.6+ MB


In [242]:
df_sales_select.to_excel(r"C:\Users\user\Downloads\Halı Satış Sorumlusu Satış For Tableau (24.03.2023-31.08.2024).xlsx", index=False)

In [244]:
df_sales_select_group_m2_product = df_sales_select.groupby(["satış sorumlusu","ürün"])["miktar m2"].sum().reset_index()
df_sales_select_group_m2_product_sort = df_sales_select_group_m2_product.sort_values(["satış sorumlusu","miktar m2"], ascending=[True,False])
df_sales_select_group_m2_product_sort = df_sales_select_group_m2_product_sort.reset_index(drop=True)
df_sales_select_group_m2_product_sort

,satış sorumlusu,ürün,miktar m2
0,ALİ YILDIRIM,MİRA 1300 GR KAYMAZ TABAN PAMUK POLYESTER ŞÖNİL,1826.68
1,ALİ YILDIRIM,LUNA 1300 GR KAYMAZ TABAN PAMUK POLYESTER ŞÖNİL,874.95
2,ALİ YILDIRIM,MARS BEYAZ DOT TABAN PELUŞ,863.46
3,ALİ YILDIRIM,"ROMA (KOTAÇMAZI 1200 GR,3MM SÜNGER,MİKROBAMBU ...",525.42
4,ALİ YILDIRIM,ORKİDE 1300 GR ÇÖKENLİ POLYESTER AKRİLİK KAYMA...,468.44
...,...,...,...
465,ŞABAN SİLAHŞÖR,LUNA 1300 GR KAYMAZ TABAN PAMUK POLYESTER ŞÖNİL,7.10
466,ŞABAN SİLAHŞÖR,NEPTÜN 1200 GR KOTAÇMAZI PAMUK TABAN PELUŞ,6.00
467,ŞABAN SİLAHŞÖR,"BERLİN (KAZAYAĞI PAMUK TABAN 750GR,SÜNGER,MİKR...",5.74
468,ŞABAN SİLAHŞÖR,"MİMOZA (450 EKO B.DOT, 3MM SÜNGER, 125 GR KUMAŞ",4.80


In [56]:
df_sales_select_group_m2_product_sort.to_excel(r"C:\Users\user\Downloads\Halı Satış Sorumlusu Ürün Satış M2 (24.03.2023-31.08.2024).xlsx", index=False)

In [246]:
df_sales_select_group_m2_model = df_sales_select.groupby(["satış sorumlusu","model"])["miktar m2"].sum().reset_index()
df_sales_select_group_m2_model_sort = df_sales_select_group_m2_model.sort_values(["satış sorumlusu","miktar m2"], ascending=[True,False])
df_sales_select_group_m2_model_sort = df_sales_select_group_m2_model_sort.reset_index(drop=True)
df_sales_select_group_m2_model_sort

,satış sorumlusu,model,miktar m2
0,ALİ YILDIRIM,MİRA,1826.68
1,ALİ YILDIRIM,LUNA,924.79
2,ALİ YILDIRIM,MARS,863.46
3,ALİ YILDIRIM,ROMA,525.42
4,ALİ YILDIRIM,ORKİDE,468.44
...,...,...,...
449,ŞABAN SİLAHŞÖR,LUNA,7.10
450,ŞABAN SİLAHŞÖR,NEPTÜN,6.00
451,ŞABAN SİLAHŞÖR,BERLİN,5.74
452,ŞABAN SİLAHŞÖR,MİMOZA,4.80


In [ ]:
df_sales_select_group_m2_model_sort.to_excel(r"C:\Users\user\Downloads\Halı Satış Sorumlusu Model Satış M2 (24.03.2023-31.08.2024).xlsx", index=False)